In [1]:
%pip install -r requirement.txt

  Obtaining dependency information for pymupdf from https://files.pythonhosted.org/packages/4e/55/43b64fa6cd048d2ea4574c045b5ac05d023254b91c2c703185f6f8a77b30/pymupdf-1.25.5-cp39-abi3-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for langchain_community from https://files.pythonhosted.org/packages/bb/72/4046a132a180b569265bc8aa7ecd6f958f6c11085bdf68c7e1bbe52f1907/langchain_community-0.3.21-py3-none-any.whl.metadata
  Obtaining dependency information for unstructured from https://files.pythonhosted.org/packages/cb/88/061a9dedd4e8cc0c31097c3275a9ef1fd7307e26afac5cd582487386e1b8/unstructured-0.17.2-py3-none-any.whl.metadata
  Obtaining dependency information for python-pptx from https://files.pythonhosted.org/packages/d9/4f/00be2196329ebbff56ce564aa94efb0fbc828d00de250b1980de1a34ab49/python_pptx-1.0.2-py3-none-any.whl.metadata
  Obtaining dependency information for aiohttp<4.0.0,>=3.8.3 from https://files.pythonhosted.org/packages/78/dc/5f3c0d27c91abf0bb5d103e9c9b0ff05

# API Key

In [4]:
# Dealing with API Key

import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


# Importing

In [8]:
import pptx # for reading pptx
from typing import List, Tuple
import fitz # for reading pdf
import unstructured
from langchain_community.document_loaders import UnstructuredPowerPointLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
folder_location = "INSERT FOLDER LOCATION"
documents: List[Tuple[str, str]] = []
# Processing texts from a folder of pptx and txt files
for filename in os.listdir(folder_location):
    filepath = os.path.join(folder_location, filename)
    if filename.endswith(".pptx"):
        try:
            loader = UnstructuredPowerPointLoader(filepath)
            loaded_docs = loader.load()
            for doc in loaded_docs:
                documents.append((doc.page_content, {"source": filename}))
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    elif filename.endswith(".txt"):
        try:
            loader = TextLoader(filepath)
            loaded_docs = loader.load()
            for doc in loaded_docs:
                documents.append((doc.page_content, {"source": filename}))
        except Exception as e:
                    print(f"Error loading {filename}: {e}")
        
    elif filename.endswith(".pdf"):
        try: 
            loader = PyPDFLoader(filepath)
            loaded_docs = loader.load()
            for doc in loaded_docs:
                documents.append((doc.page_content, {"source": filename}))
        except Exception as e:
            print(f"Error loading {filename}: {e}")
            
    else:
        print("Only .pdf, .txt, and .pptx files are allowed!")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.create_documents([doc[0] for doc in documents], metadatas=[doc[1] for doc in documents])